To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [109]:
print("yoloooo test that sheet hehe")

yoloooo test that sheet hehe


In [2]:
import torch
print(torch.cuda.is_available())           # Doit être True
print(torch.cuda.get_device_name(0))       # Doit afficher "NVIDIA GeForce RTX 3070 Ti"

True
NVIDIA GeForce RTX 3070 Ti


### Unsloth

In [144]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.4.7: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3070 Ti. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

OSError: unsloth/mistral-7b-bnb-4bit does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `ChatML` format for conversation style finetunes. We use [Open Assistant conversations](https://huggingface.co/datasets/philschmid/guanaco-sharegpt-style) in ShareGPT style. ChatML renders multi turn conversations like below:

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What's the capital of France?<|im_end|>
<|im_start|>assistant
Paris.
```

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old` and our own optimized `unsloth` template.

Normally one has to train `<|im_start|>` and `<|im_end|>`. We instead map `<|im_end|>` to be the EOS token, and leave `<|im_start|>` as is. This requires no additional training of additional tokens.

Note ShareGPT uses `{"from": "human", "value" : "Hi"}` and not `{"role": "user", "content" : "Hi"}`, so we use `mapping` to map it.

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
from unsloth.chat_templates import get_chat_template
"""
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("philschmid/guanaco-sharegpt-style", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# ultrachat_train = df
"""

In [23]:
# For google collab only
from google.colab import files
import pandas as pd

# Télécharger les fichiers depuis l'ordinateur local
eval_dataset = files.upload()

# Lire les fichiers JSONL téléchargés
for filename in eval_dataset.keys():
    if filename.endswith('.jsonl'):
        df = pd.read_json(filename, lines=True)
        print(f"Data from {filename} loaded successfully")
        print(df.head())


Saving ultrachat_second_selection_training_dataset.jsonl to ultrachat_second_selection_training_dataset (1).jsonl
Data from ultrachat_second_selection_training_dataset (1).jsonl loaded successfully
                                            messages
0  [{'role': 'user', 'content': '{
  "Chapter dat...
1  [{'role': 'user', 'content': '{
  "Chapter dat...
2  [{'role': 'user', 'content': '{
  "Chapter dat...
3  [{'role': 'user', 'content': '{
  "Chapter dat...
4  [{'role': 'user', 'content': '{
  "Chapter dat...


In [110]:
# For local usage only
import pandas as pd
ultrachat_train = pd.read_json(r'C:\Users\amande\Desktop\Pierre\TI_github\Terra_Inco\data\ultrachat_simplified_second_selection_training_dataset.jsonl', lines=True)

In [ ]:
#ultrachat_train = load_dataset("json", data_files="C:/Users/amande/Desktop/Pierre/TI_github/Terra_Inco/data/ultrachat_second_selection_training_dataset.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Let's see how the `ChatML` format works by printing the 5th element

In [111]:
import numpy as np
print(np.shape(dataset))
print(np.shape(ultrachat_train))

(9033, 2)
(47, 1)


In [30]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 9033
})

In [112]:
ultrachat_train

,messages
0,"[{'role': 'user', 'content': '{ ""Chapter dat..."
1,"[{'role': 'user', 'content': '{ ""Chapter dat..."
2,"[{'role': 'user', 'content': '{ ""Chapter dat..."
3,"[{'role': 'user', 'content': '{ ""Chapter dat..."
4,"[{'role': 'user', 'content': '{ ""Chapter dat..."
5,"[{'role': 'user', 'content': '{ ""Chapter dat..."
6,"[{'role': 'user', 'content': '{ ""Chapter dat..."
7,"[{'role': 'user', 'content': '{ ""Chapter dat..."
8,"[{'role': 'user', 'content': '{ ""Chapter dat..."
9,"[{'role': 'user', 'content': '{ ""Chapter dat..."


In [ ]:
# Not the good format
"""
from datasets import Dataset
exploded_df = ultrachat_train.explode('messages').reset_index()
normalized_df = pd.json_normalize(exploded_df['messages'])
normalized_df['index'] = exploded_df['index']
pivoted_df = normalized_df.pivot(index='index', columns='role', values='content').reset_index(drop=True)
print(pivoted_df.head())
pivoted_dataset = Dataset.from_pandas(pivoted_df)
"""

role                                          assistant  \
0     [Sidenote: A Counterblaste to James I.]\n\nSo ...   
1     CHAPTER X\n\nDOWN A BIG HOLE\n\n\nFor a few se...   
2     18\n\n\nCaptain Sir Henry Quill opened the doo...   
3     CHAPTER IX\n\nMR. THOMAS MARVEL\n\n\nYou must ...   
4     XXIX\n\n\nGorham received two callers on that ...   

role                                               user  
0     {\n  "Chapter data": {\n    "Book": 9468,\n   ...  
1     {\n  "Chapter data": {\n    "Book": 2498,\n   ...  
2     {\n  "Chapter data": {\n    "Book": 4983,\n   ...  
3     {\n  "Chapter data": {\n    "Book": 8462,\n   ...  
4     {\n  "Chapter data": {\n    "Book": 2011,\n   ...  


In [124]:
from datasets import Dataset

# Renommage simple selon format Alpaca
pivoted_df["instruction"] = "Write a complete chapter based on the following summary:"
pivoted_df["input"] = pivoted_df["user"]
pivoted_df["output"] = pivoted_df["assistant"]

alpaca_df = pivoted_df[["instruction", "input", "output"]]

# Convertir en Dataset HuggingFace
alpaca_dataset = Dataset.from_pandas(alpaca_df)


In [117]:
alpaca_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 47
})

In [119]:
pivoted_df

role,assistant,user,instruction,input,output
0,[Sidenote: A Counterblaste to James I.]\n\nSo ...,"{\n ""Chapter data"": {\n ""Book"": 9468,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 9468,\n ...",[Sidenote: A Counterblaste to James I.]\n\nSo ...
1,CHAPTER X\n\nDOWN A BIG HOLE\n\n\nFor a few se...,"{\n ""Chapter data"": {\n ""Book"": 2498,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 2498,\n ...",CHAPTER X\n\nDOWN A BIG HOLE\n\n\nFor a few se...
2,18\n\n\nCaptain Sir Henry Quill opened the doo...,"{\n ""Chapter data"": {\n ""Book"": 4983,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 4983,\n ...",18\n\n\nCaptain Sir Henry Quill opened the doo...
3,CHAPTER IX\n\nMR. THOMAS MARVEL\n\n\nYou must ...,"{\n ""Chapter data"": {\n ""Book"": 8462,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 8462,\n ...",CHAPTER IX\n\nMR. THOMAS MARVEL\n\n\nYou must ...
4,XXIX\n\n\nGorham received two callers on that ...,"{\n ""Chapter data"": {\n ""Book"": 2011,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 2011,\n ...",XXIX\n\n\nGorham received two callers on that ...
5,II\n\nTHE ORIGIN OF ENERGY\n\n\nI\n\nWhat has ...,"{\n ""Chapter data"": {\n ""Book"": 1283,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 1283,\n ...",II\n\nTHE ORIGIN OF ENERGY\n\n\nI\n\nWhat has ...
6,"CHAPTER XII\n\nPELTED BY HAILSTONES\n\n\n""Yes,...","{\n ""Chapter data"": {\n ""Book"": 8031,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 8031,\n ...","CHAPTER XII\n\nPELTED BY HAILSTONES\n\n\n""Yes,..."
7,7\n\n\nThe burly character who had appeared at...,"{\n ""Chapter data"": {\n ""Book"": 4226,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 4226,\n ...",7\n\n\nThe burly character who had appeared at...
8,"CHAPTER XVI\n\nMRS. VRAIN AT BAY\n\n\n""You saw...","{\n ""Chapter data"": {\n ""Book"": 3398,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 3398,\n ...","CHAPTER XVI\n\nMRS. VRAIN AT BAY\n\n\n""You saw..."
9,"CHAPTER XVIII\n\nA SEPARATION\n\n\n""Mr. Hardle...","{\n ""Chapter data"": {\n ""Book"": 1362,\n ...",Write a complete chapter based on these data.,"{\n ""Chapter data"": {\n ""Book"": 1362,\n ...","CHAPTER XVIII\n\nA SEPARATION\n\n\n""Mr. Hardle..."


In [120]:
print(pivoted_df["assistant"])
print(pivoted_df["user"])

0     [Sidenote: A Counterblaste to James I.]\n\nSo ...
1     CHAPTER X\n\nDOWN A BIG HOLE\n\n\nFor a few se...
2     18\n\n\nCaptain Sir Henry Quill opened the doo...
3     CHAPTER IX\n\nMR. THOMAS MARVEL\n\n\nYou must ...
4     XXIX\n\n\nGorham received two callers on that ...
5     II\n\nTHE ORIGIN OF ENERGY\n\n\nI\n\nWhat has ...
6     CHAPTER XII\n\nPELTED BY HAILSTONES\n\n\n"Yes,...
7     7\n\n\nThe burly character who had appeared at...
8     CHAPTER XVI\n\nMRS. VRAIN AT BAY\n\n\n"You saw...
9     CHAPTER XVIII\n\nA SEPARATION\n\n\n"Mr. Hardle...
10    _WHEN_ Mr. Dinneford and the policeman sent by...
11    CHAPTER VIII\n\n_White's Brain--Yellow's Head_...
12    XXV\n\n\nA week later the Gorhams' dinner-tabl...
13    CHAPTER XI\n\nDEAR MAMMA\n\n\nBERYL must have ...
14    CHAPTER 12\n\n\n"TOWARDS midnight Mr. Venables...
15    CHAPTER XIX\n\nA FIENDISH MACHINE\n\n\nA chill...
16    XXII\n\nYET THREE DAYS\n\n\nEsmay sat in the g...
17    Chapter V.\n\n"Follow me," said the captai

In [121]:
print(np.shape(dataset))
print(type(dataset))
print(np.shape(alpaca_dataset))
print(type(alpaca_dataset))

(9033, 2)
<class 'datasets.arrow_dataset.Dataset'>
(47, 3)
<class 'datasets.arrow_dataset.Dataset'>


In [52]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 9033
})

In [51]:
pivoted_dataset

Dataset({
    features: ['assistant', 'user'],
    num_rows: 32
})

IMPORTANT NOTES: if I want a shareGPT format as the dataset in the tuto I need to transfo my format here and then use the #dataset = dataset.map(formatting_prompts_func, batched = True,)
Here i don't care about multi-turns conversation so I keep my "user" / "assistant" format

If you're looking to make your own chat template, that also is possible! You must use the Jinja templating regime. We provide our own stripped down version of the `Unsloth template` which we find to be more efficient, and leverages ChatML, Zephyr and Alpaca styles.

More info on chat templates on [our wiki page!](https://github.com/unslothai/unsloth/wiki#chat-templates)

In [51]:
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

Here let's add a step of conversion. I need to convert my dataset Input / Output should be not in a Json format but in a clear and narrative way.

In [ ]:
# ...


In [125]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "alpaca",
    map_eos_token = True,
)

def format_alpaca(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
    }

"""
def format_alpaca(example):
    return {
        "text": f"### Input:\nChapter {example['Chapter data']['Chapter_ID']} of Book {example['Chapter data']['Book']}\n\nEvents:\n" + 
                "\n".join([f"- {event['event']}: {event['details']}" for event in example['events_description']]) +
                "\n\nPlot Evolution:\nInitial Situation: {example['plot_evolution']['initial_situation']}\nSituation at the end of the chapter: {example['plot_evolution']['situation_at_the_end_of_the_chapter']}" +
                "\n\nCharacter Evolution:\n" + 
                "\n".join([f"{character['character']} - Development: {character['development']}" for character in example['characters_evolution']])
    }
"""
alpaca_dataset = alpaca_dataset.map(format_alpaca)




Map:   0%|          | 0/47 [00:00<?, ? examples/s]

In [126]:
print(alpaca_dataset[0])

{'instruction': 'Write a complete chapter based on the following summary:', 'input': '{\n  "Chapter data": {\n    "Book": 9468,\n    "Chapter_ID": 64\n  },\n  "events_description": [\n    {\n      "event": "Introduction of Tobacco",\n      "details": "The author begs mercy on behalf of tobacco, which has been condemned by King James I in his work \'A Counterblaste to Tobacco\'."\n    },\n    {\n      "event": "Critique of the King\'s Arguments",\n      "details": "The author criticizes the king\'s arguments against tobacco, calling them \'asinine\' and pointing out logical fallacies."\n    },\n    {\n      "event": "Response to Specific Arguments",\n      "details": "The author responds to four of the king\'s arguments: (1) The misinterpretation of a physiological fact, (2) The fallacy that people\'s affection for monarchs proves their goodness, (3) The claim that tobacco wastes one\'s substance, and (4) The association of tobacco smoke with hellfire."\n    },\n    {\n      "event": "C

In [104]:
np.shape(alpaca_dataset)

(32, 4)

In [127]:
alpaca_dataset['text']

['### Instruction:\nWrite a complete chapter based on the following summary:\n\n### Input:\n{\n  "Chapter data": {\n    "Book": 9468,\n    "Chapter_ID": 64\n  },\n  "events_description": [\n    {\n      "event": "Introduction of Tobacco",\n      "details": "The author begs mercy on behalf of tobacco, which has been condemned by King James I in his work \'A Counterblaste to Tobacco\'."\n    },\n    {\n      "event": "Critique of the King\'s Arguments",\n      "details": "The author criticizes the king\'s arguments against tobacco, calling them \'asinine\' and pointing out logical fallacies."\n    },\n    {\n      "event": "Response to Specific Arguments",\n      "details": "The author responds to four of the king\'s arguments: (1) The misinterpretation of a physiological fact, (2) The fallacy that people\'s affection for monarchs proves their goodness, (3) The claim that tobacco wastes one\'s substance, and (4) The association of tobacco smoke with hellfire."\n    },\n    {\n      "even

In [128]:
print(alpaca_dataset.column_names)

['instruction', 'input', 'output', 'text']


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
#tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")


In [129]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = alpaca_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,  #C’est pour le multithread dans .map()
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/47 [00:00<?, ? examples/s]

In [131]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3070 Ti. Max memory = 8.0 GB.
4.105 GB of memory reserved.


In [132]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 47 | Num Epochs = 10 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 19,611,648/2,000,000,000 (0.98% trained)


Step,Training Loss
1,0.203300
2,0.005500
3,0.006900
4,0.006600
5,0.005200
6,0.004200
7,0.004000
8,0.003100
9,0.003200
10,0.002500


In [133]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

552.9158 seconds used for training.
9.22 minutes used for training.
Peak reserved memory = 4.105 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 51.313 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! Since we're using `ChatML`, use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [56]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>user\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>\n<|im_start|>assistant\nThe next number in the Fibonacci sequence is 13. This is calculated by adding the last two numbers in the sequence together: 8 + 5 = 13. The Fibonacci sequence continues in this way, with each number being the sum of the two numbers before it.<|im_end|>']

In my cas I use the Alpaca format

In [141]:
FastLanguageModel.for_inference(model)

instruction = "Write a complete chapter based on the following summary:"
input_text = "Chapter data\": {\n    \"Book\": 2498,\n    \"Chapter_ID\": 10\n  },\n  \"events_description\": [\n    {\n      \"event\": \"Nan is stuck on a swing.\",\n      \"details\": \"Nan is stuck on the swing at her friends house and needs help getting off.\"\n    },\n    {\n      \"event\": \"Freddie finds eggs in the barn.\",\n      \"details\": \"Flossie and Freddie are gathering eggs in the barn. Flossie found some eggs first, and then Freddie went to look for his own nest of eggs.\"\n    },\n    {\n      \"event\": \"Freddie falls into a hole in the haymow.\",\n      \"details\": \"While searching for eggs in the haymow, Freddie slides down a pile of hay and falls into a big hole. He calls out for Nan to help him.\"\n    }\n  ],\n  \"plot_evolution\": {\n    \"initial_situation\": \"The Bobbsey twins are visiting their cousins at Meadow Brook farm. They are excited about the upcoming Bolton County Fair and plan to go fishing for bullfrogs.\",\n    \"situation_at_the_end_of_the_chapter\": \"Nan is helping Flossie gather eggs in the barn while Freddie goes off on his own to find more eggs. Freddie climbs up to the haymow and falls into a hole, calling out for Nan's help.\"\n  },\n  \"characters_evolution\": [\n    {\n      \"character\": \"Nan\",\n      \"development\": \"Nan shows responsibility by helping her mother unpack and then taking care of Flossie and Freddie. She also demonstrates patience and understanding in dealing with the younger twins.\"\n    },\n    {\n      \"character\": \"Freddie\",\n      \"development\": \"Freddie displays curiosity and independence by going off on his own to find eggs, but ends up needing help after falling into a hole."
prompt = f"{instruction}\n{input_text}"


In [ ]:
prompt = (
    "### Instruction:"
    "Write a complete chapter based on the following summary:"
    "In a future where Earth is in ruins, a handful of survivors struggle to preserve humanity. The protagonist, Elena, a brilliant yet solitary scientist, discovers an ancient technology that could reverse the devastating effects on the environment. However, this discovery attracts the attention of powerful corporations who seek to use it to dominate the last remnants of civilization. With the help of a diverse group of rebels, Elena must decide how far she is willing to go to save what remains of humanity, while confronting the ethical dilemmas surrounding her invention. The novel explores themes of sacrifice, love, and responsibility in the face of the unknown."
)

In [142]:
prompt

'Write a complete chapter based on the following summary:\nChapter data": {\n    "Book": 2498,\n    "Chapter_ID": 10\n  },\n  "events_description": [\n    {\n      "event": "Nan is stuck on a swing.",\n      "details": "Nan is stuck on the swing at her friends house and needs help getting off."\n    },\n    {\n      "event": "Freddie finds eggs in the barn.",\n      "details": "Flossie and Freddie are gathering eggs in the barn. Flossie found some eggs first, and then Freddie went to look for his own nest of eggs."\n    },\n    {\n      "event": "Freddie falls into a hole in the haymow.",\n      "details": "While searching for eggs in the haymow, Freddie slides down a pile of hay and falls into a big hole. He calls out for Nan to help him."\n    }\n  ],\n  "plot_evolution": {\n    "initial_situation": "The Bobbsey twins are visiting their cousins at Meadow Brook farm. They are excited about the upcoming Bolton County Fair and plan to go fishing for bullfrogs.",\n    "situation_at_the_e

In [143]:
inputs = tokenizer(
    prompt,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=200,
    use_cache=True,    
    temperature=0.7,
    top_p=0.95,
    do_sample=True,
)


<bos>Write a complete chapter based on the following summary:
Chapter data": {
    "Book": 2498,
    "Chapter_ID": 10
  },
  "events_description": [
    {
      "event": "Nan is stuck on a swing.",
      "details": "Nan is stuck on the swing at her friends house and needs help getting off."
    },
    {
      "event": "Freddie finds eggs in the barn.",
      "details": "Flossie and Freddie are gathering eggs in the barn. Flossie found some eggs first, and then Freddie went to look for his own nest of eggs."
    },
    {
      "event": "Freddie falls into a hole in the haymow.",
      "details": "While searching for eggs in the haymow, Freddie slides down a pile of hay and falls into a big hole. He calls out for Nan to help him."
    }
  ],
  "plot_evolution": {
    "initial_situation": "The Bobbsey twins are visiting their cousins at Meadow Brook farm. They are excited about the upcoming Bolton County Fair and plan to go fishing for bullfrogs.",
    "situation_at_the_end_of_the_chapter

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [57]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant
The next number in the Fibonacci sequence is 13. This is calculated by adding the last two numbers in the sequence together: 8 + 5 = 13. The Fibonacci sequence continues in this way, with each number being the sum of the two numbers before it.<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [58]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [59]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<|im_start|>user
What is a famous tall tower in Paris?<|im_end|>
<|im_start|>assistant
The Eiffel Tower is a famous tall tower in Paris. It is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. The Eiffel Tower is one of the most recognizable structures in the world and is a popular tourist destination. It is also a symbol of France and is often used as a landmark for navigation.<|im_end|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoModelForPeftCausalLM
    from transformers import AutoTokenizer

    model = AutoModelForPeftCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
